In [1]:
# clone the repo
!git clone https://github.com/<Anonymized>/WSQASA

Cloning into 'WSQASA'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 72 (delta 24), reused 69 (delta 21), pack-reused 0
Unpacking objects: 100% (72/72), 24.62 MiB | 4.24 MiB/s, done.


In [23]:
### parameters for all pipes
dataset_name = "tweet_qa"

main_path = "/content/WSQASA"
scripts_path = f"{main_path}/scripts/pipe.zip"
input_file_issues_path = f'{main_path}/data/LID_all_issues.csv'

dir_path = f'{main_path}/'
data_dir_path = f'{dir_path}data/'
model_dir_path = f'{dir_path}models/'
input_file_train_data = f'{main_path}/data/train_data.pkl'
random_seed = "42"

In [24]:
import os 
if not os.path.exists(main_path):
  os.mkdir(main_path)

if not os.path.exists(dir_path):
  os.mkdir(dir_path)

In [25]:
### notebook 1 parameters ###
min_thold = "0.75"
min_sentiment_thold = "-0.1"
search_model_name = "all-MiniLM-L6-v2"
output_file_path_1 = f'{main_path}/data/syntatic_train_with_similarity.pkl'

In [26]:
### notebook 2 parameters ###
input_file_path_2 = f'{data_dir_path}{dataset_name}_validation_form.pkl'
output_file_path_2 = f'{data_dir_path}{dataset_name}_test_with_similarity.pkl'

In [27]:
### notebook 3 parameters ###
domain = 'negative'
input_file_path_3 = output_file_path_1
output_model_path_3 = f'{model_dir_path}not_{dataset_name}_finetuned_{domain}_model'

# sentiment models parameters #
model_checkpoint = "deepset/bert-base-cased-squad2"
batch_size = "16"

# tokenizer
max_length = "512"  # The maximum length of a feature (question and context)
doc_stride = "128"  # The allowed overlap between two part of the context when splitting is performed.

# hyper-parameters 
learning_rate = "2e-5"
num_train_epochs = "1"
weight_decay = "0.01"
encoder_layers_to_freeze = "0"

In [28]:
### notebook 4 parameters ###
input_file_path_4 = output_file_path_2
input_model_path_4 = f'{model_dir_path}not_{dataset_name}_finetuned_{domain}_model'

output_file_path_4 = f'{data_dir_path}{dataset_name}_validation_{domain}_with_finetuned_predictions.pkl'

model_tokenizer = "deepset/bert-base-cased-squad2"

In [29]:
### notebook 4.1 parameters ###
input_file_path_4_1 = output_file_path_2
output_file_path_4_1 = f'{data_dir_path}{dataset_name}_validation_{domain}_with_not_finetuned_predictions.pkl'

In [30]:
#### notebook 5 parameters ###
input_file_path_not_finetuned_5 = output_file_path_4_1
input_file_path_finetuned_5 = output_file_path_4

output_file_path_5 = f'{data_dir_path}{dataset_name}_metrics.csv'

In [31]:
import os
# base variables 
os.environ['dataset_name'] = dataset_name
os.environ['dir_path'] = dir_path
os.environ['data_dir_path'] = data_dir_path
os.environ['model_dir_path'] = model_dir_path
os.environ['input_file_issues_path'] = input_file_issues_path
os.environ['input_file_train_data'] = input_file_train_data
os.environ['random_seed'] = random_seed

## search variables - notebook 1,2
os.environ['min_thold'] = min_thold
os.environ['min_sentiment_thold'] = min_sentiment_thold
os.environ['search_model_name'] = search_model_name
os.environ['output_file_path_1'] = output_file_path_1
os.environ['input_file_path_2'] = input_file_path_2
os.environ['output_file_path_2'] = output_file_path_2

## notebook 3 parameters train !&A models
os.environ['domain'] = domain
os.environ['input_file_path_3'] = input_file_path_3
os.environ['output_model_path_3'] = output_model_path_3
os.environ['model_checkpoint'] = model_checkpoint
os.environ['batch_size'] = batch_size
os.environ['max_length'] = max_length
os.environ['doc_stride'] = doc_stride
os.environ['learning_rate'] = learning_rate
os.environ['num_train_epochs'] = num_train_epochs
os.environ['weight_decay'] = weight_decay
os.environ['encoder_layers_to_freeze'] = encoder_layers_to_freeze

### notebook 4 parameters ###
os.environ['input_file_path_4'] = input_file_path_4
os.environ['input_model_path_4'] = input_model_path_4
os.environ['output_file_path_4_1'] = output_file_path_4_1
os.environ['output_file_path_4'] = output_file_path_4
os.environ['input_file_path_4_1'] = input_file_path_4_1
os.environ['model_tokenizer'] = model_tokenizer

### notebooks 5 parameters ###
os.environ['input_file_path_not_finetuned_5'] = input_file_path_not_finetuned_5
os.environ['input_file_path_finetuned_5'] = input_file_path_finetuned_5
os.environ['output_file_path_5'] = output_file_path_5

In [11]:
%%capture
## notebook 1 dependencies ##
!pip install faiss-gpu==1.7.2
!pip install sentence-transformers==2.2.2
!pip install vaderSentiment==3.3.2
!pip install -U tensorflow==2.10 

## notebook 1 dependencies ##
!pip install transformers==4.22.2 datasets==2.5.1 

## notebook 2 dependencies
!pip install huggingface_hub==0.10.0

## notebook 3 dependencies
!pip install evaluate==0.2.2

## notebook 5 dependencies
!pip install tensorflow_probability==0.12.2

In [12]:
!pip freeze > requerements.txt

In [17]:
!unzip "$scripts_path"

Archive:  /content/WSQASA/scripts/pipe.zip
  inflating: 2_1_similarity_search_answer_test_df.py  
  inflating: 2_similarity_search_answers_artificial_df.py  
  inflating: 3_training_the_qa_model_on_syntatic_data.py  
  inflating: 4_1_run_not_finetuned_model_over_dataset_with_specific_domain.py  
  inflating: 4_run_model_over_dataset_with_specific_domain.py  
  inflating: 5_compare_performance_of_models.py  


### 1. Check if the notebook train data with similarity already exists, if not filters the train dataset via sentiment and similarity filtering

In [18]:
if not os.path.exists(os.environ['output_file_path_1']):
  !python '2_similarity_search_answers_artificial_df.py'  

[INFO] dataset_name :  tweet_qa
[INFO] min_thold :  0.75
[INFO] min_sentiment_thold :  -0.1
[INFO] dir_path :  /content/WSQASA//
[INFO] data_dir_path :  /content/WSQASA//data/
[INFO] model_dir_path :  /content/WSQASA//models/
[INFO] input_file_path :  /content/WSQASA//data/train_data.pkl
[INFO] input_file_issues_path :  /content/WSQASA//data/LID_all_issues.csv
[INFO] output_file_path :  /content/WSQASA//data/syntatic_train_with_similarity.pkl
[INFO] model_name :  all-MiniLM-L6-v2
Downloading: 100% 1.18k/1.18k [00:00<00:00, 1.02MB/s]
Downloading: 100% 190/190 [00:00<00:00, 190kB/s]
Downloading: 100% 10.6k/10.6k [00:00<00:00, 7.19MB/s]
Downloading: 100% 612/612 [00:00<00:00, 565kB/s]
Downloading: 100% 116/116 [00:00<00:00, 83.1kB/s]
Downloading: 100% 39.3k/39.3k [00:00<00:00, 161kB/s]
Downloading: 100% 90.9M/90.9M [00:01<00:00, 61.0MB/s]
Downloading: 100% 53.0/53.0 [00:00<00:00, 37.6kB/s]
Downloading: 100% 112/112 [00:00<00:00, 80.2kB/s]
Downloading: 100% 466k/466k [00:00<00:00, 1.85MB/s

## 2. Create the test set based on the similarity

In [36]:
!python '2_1_similarity_search_answer_test_df.py'
print("\n finished test similairity ")

[INFO] dataset_name :  tweet_qa
[INFO] min_thold :  0.75
[INFO] min_sentiment_thold :  -0.1
[INFO] dir_path :  /content/WSQASA/
[INFO] data_dir_path :  /content/WSQASA/data/
[INFO] model_dir_path :  /content/WSQASA/models/
[INFO] input_file_path :  /content/WSQASA/data/tweet_qa_validation_form.pkl
[INFO] input_file_issues_path :  /content/WSQASA/data/LID_all_issues.csv
[INFO] output_file_path :  /content/WSQASA/data/tweet_qa_test_with_similarity.pkl
[INFO] model_name :  all-MiniLM-L6-v2
Batches: 100% 2001/2001 [00:33<00:00, 59.95it/s]
Batches: 100% 45/45 [00:00<00:00, 73.32it/s]
100% 1177/1177 [00:00<00:00, 33215.78it/s]
job success



 finished test similairity 


## 3 Train the Q&A model

In [ ]:
import time
time.sleep(2)

In [37]:
!python '3_training_the_qa_model_on_syntatic_data.py'  
print("\n finished train with syntatic data")

 [INFO] dataset_name :  tweet_qa
 [INFO] domain :  negative
 [INFO] random_seed :  42
 [INFO] dir_path :  /content/WSQASA/
 [INFO] data_dir_path :  /content/WSQASA/data/
 [INFO] model_dir_path :  /content/WSQASA/models/
 [INFO] input_file_path :  /content/WSQASA/data/syntatic_train_with_similarity.pkl
 [INFO] output_model_path :  /content/WSQASA/models/not_tweet_qa_finetuned_negative_model
 [INFO] model_checkpoint :  deepset/bert-base-cased-squad2
 [INFO] batch_size :  16
 [INFO] max_length :  512
 [INFO] doc_stride :  128
 [INFO] learning_rate :  2e-05
 [INFO] num_train_epochs :  1
 [INFO] weight_decay :  0.01
2023-04-18 03:29:50.590197: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 03:29:50.766403: E t

## 4. Run finetuned model over the dataset

In [38]:
import time
time.sleep(2)

In [39]:
!python '4_run_model_over_dataset_with_specific_domain.py'
print("\n finished trained model run over with test data")

 [INFO] dataset_name :  tweet_qa
 [INFO] domain :  negative
 [INFO] dir_path :  /content/WSQASA/
 [INFO] data_dir_path :  /content/WSQASA/data/
 [INFO] model_dir_path :  /content/WSQASA/models/
 [INFO] input_file_path :  /content/WSQASA/data/tweet_qa_test_with_similarity.pkl
 [INFO] input_model_path :  /content/WSQASA/models/not_tweet_qa_finetuned_negative_model
 [INFO] output_file_path :  /content/WSQASA/data/tweet_qa_validation_negative_with_finetuned_predictions.pkl
 [INFO] model_tokenizer :  deepset/bert-base-cased-squad2
 [INFO] max_length :  512
 [INFO] doc_stride :  128
 [INFO] batch_size :  16
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
Moving 0 files to the new cache system
0it [00:00, ?it/s]
2023-04-18 03:40:24.286926: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow bina

## 4.1. Run not finetuned model over the dataset

In [40]:
!python '4_1_run_not_finetuned_model_over_dataset_with_specific_domain.py'
print("\n finished not trained model run over with test data")

 [INFO] dataset_name :  tweet_qa
 [INFO] domain :  negative
 [INFO] dir_path :  /content/WSQASA/
 [INFO] data_dir_path :  /content/WSQASA/data/
 [INFO] input_file_path :  /content/WSQASA/data/tweet_qa_test_with_similarity.pkl
 [INFO] output_file_path :  /content/WSQASA/data/tweet_qa_validation_negative_with_not_finetuned_predictions.pkl
 [INFO] model_name :  deepset/bert-base-cased-squad2
 [INFO] max_length :  512
 [INFO] doc_stride :  128
 [INFO] batch_size :  16
2023-04-18 03:40:36.891540: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 03:40:37.075916: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alr

## 5. Generate Metrics

In [41]:
import time
time.sleep(2)

In [42]:
!python '5_compare_performance_of_models.py'
print("\n finished not trained model run over with test data")

 [INFO] dataset_name :  tweet_qa
 [INFO] domain :  negative
 [INFO] dir_path :  /content/WSQASA/
 [INFO] data_dir_path :  /content/WSQASA/data/
 [INFO] input_file_path_not_finetuned :  /content/WSQASA/data/tweet_qa_validation_negative_with_not_finetuned_predictions.pkl
 [INFO] input_file_path_finetuned :  /content/WSQASA/data/tweet_qa_validation_negative_with_finetuned_predictions.pkl
 [INFO] output_file_path :  /content/WSQASA/data/tweet_qa_metrics.csv
2023-04-18 03:40:52.434225: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 03:40:52.623606: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been r

In [43]:
import pandas as pd
metrics = pd.read_csv(os.environ['output_file_path_5'])
metrics['total_train_data'] = len(pd.read_pickle(output_file_path_1)['dataset']!= dataset_name)
metrics.to_csv(os.environ['output_file_path_5'])
metrics

,dataset,model_name,exact_match,f1,bleu_1_ngram,bleu_2_ngram,bleu_3_ngram,bleu_4_ngram,meteor,total_test_data,total_train_data
0,tweet_qa,not finetuned,0.510753,0.694614,0.417197,0.342114,0.279771,0.224116,0.576341,186,3257
1,tweet_qa,finetuned,0.650538,0.795524,0.577528,0.501968,0.437936,0.377177,0.600162,186,3257
